In [1]:
import spacy
import pickle,random
from tqdm import tqdm
import os
import sys,fitz
import pandas as pd
nlp = spacy.blank('en')
# i used spacy - 2.3.7 for spacy 3 the training would in different manners

In [22]:
print(spacy.__version__)

2.3.7


In [31]:
print('current working directory is:',os.getcwd())

current working directory is: C:\Users\Ravi kumar\Documents


### Pipeline functions

In [2]:
def read_file(path):
    train_data1 = pickle.load(open(path,'rb'))
    return train_data1

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        print('first if..')
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
        
    for _,annotation in train_data:
        for ent in annotation['entities']:
            #print('second for',ent)
            ner.add_label(ent[2])
    
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe!='ner']
    print(other_pipes)
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(30):
            print('starting iteration:',str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text,annotations in train_data:
                try:
                    nlp.update(
                    [text],
                    [annotations],
                    drop=0.2,
                    sgd=optimizer,
                    losses=losses)
                except Exception as e:
                    pass
            print(losses)
    return nlp
            
def model_saving(model_name,nlp):
    nlp.to_disk(model_name)
    
    
def load_model(model_name):
    nlp_model = spacy.load(model_name)
    return nlp_model
    
    

def prediction(nlp_model,pred_text):
    doc = nlp_model(pred_text)
    cols = []
    data = []
    for ent in doc.ents:
        print(f"{ent.label_.upper():{30}}-{ent.text}")
        cols.append(ent.label_.upper())
        data.append(ent.text)
    em_df = pd.DataFrame(data,index=cols)
    return em_df.to_dict()

## Loading custome created data from a pickle file

In [5]:
data = read_file("train_data.pkl")
data[:2]

[('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex

### Training custom spacy NER model with our data

In [6]:
nlp = train_model(data)

first if..
[]
starting iteration: 0


C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Harini Komaravelli Test Analyst at Oracle, Hyderab..." with entities "[(2275, 2281, 'Companies worked at'), (2235, 2241,...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the ali

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Bangalore Tavarekere Volunteer Contestant, Yappon ..." with entities "[(386, 391, 'Graduation Year'), (320, 361, 'Colleg...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Palani S Senior Technology Support Executive at In..." with entities "[(3660, 3664, 'Graduation Year'), (3598, 3602, 'Gr...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Divesh Singh Bengaluru, Karnataka - Email me on In..." with entities "[(947, 1180, 'Skills'), (833, 845, 'College Name')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Pranay Sathu Software Test Automation Engineer  Hy..." with entities "[(2051, 2567, 'Skills'), (1914, 1943, 'College Nam...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Sayani Goswami I Phone solution Consultant  Kolkat..." with entities "[(314, 333, 'College Name'), (261, 271, 'Companies...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Ramya. P Hyderabad, Telangana - Email me on Indeed..." with entities "[(4542, 4549, 'Skills'), (4178, 4187, 'Skills'), (...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

...

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Sai Patha Mule ESB Integration Developer - Cisco S..." with entities "[(11921, 12599, 'Skills'), (11791, 11876, 'Skills'...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Arpit Jain Quality Analyst - ThoughtWorks Technolo..." with entities "[(2301, 2380, 'Skills'), (2088, 2131, 'College Nam...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Yogi Pesaru Developer - Infosys Limited  Hyderabad..." with entities "[(3748, 3809, 'Skills'), (3679, 3745, 'Skills'), (...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Suresh Kanagala Architecture SharePoint/Office 365..." with entities "[(1171, 1574, 'Skills'), (962, 1096, 'Skills'), (9...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Shraddha Achar Mathura, Uttar Pradesh - Email me o..." with entities "[(975, 1020, 'Skills'), (814, 843, 'College Name')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Soumya Balan IT SUPPORT  Sulthan Bathery, Kerala, ..." with entities "[(3913, 4370, 'Skills'), (3880, 3884, 'Graduation ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Sai Vivek Venkatraman Decisive, Data driven and re..." with entities "[(5105, 5109, 'Graduation Year'), (5043, 5095, 'Co...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Puneet Singh Associate Software Engineer  Bengalur..." with entities "[(989, 1007, 'Skills'), (952, 968, 'Skills'), (919...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Ravi kumar\AppData\Roam

{'ner': 13861.460348474411}
starting iteration: 1
{'ner': 11330.502714037284}
starting iteration: 2
{'ner': 8007.790471395832}
starting iteration: 3
{'ner': 6736.12221712871}
starting iteration: 4
{'ner': 6114.9318628080855}
starting iteration: 5
{'ner': 5255.057870372597}
starting iteration: 6
{'ner': 6244.724191043187}
starting iteration: 7
{'ner': 4780.24919755004}
starting iteration: 8
{'ner': 4696.894534563914}
starting iteration: 9
{'ner': 4692.833066708723}
starting iteration: 10
{'ner': 4253.12616623784}
starting iteration: 11
{'ner': 4190.2835630332675}
starting iteration: 12
{'ner': 4481.404220885857}
starting iteration: 13
{'ner': 4242.903650870667}
starting iteration: 14
{'ner': 3304.936673809313}
starting iteration: 15
{'ner': 3368.7648822801307}
starting iteration: 16
{'ner': 3679.565749784107}
starting iteration: 17
{'ner': 3370.1390585690574}
starting iteration: 18
{'ner': 3828.0688127706917}
starting iteration: 19
{'ner': 3294.559037205477}
starting iteration: 20
{'ner

### Saving trained model to current working directory we can also save to deired path

In [12]:
model_saving('nlp_model',nlp)

### Now loading model for prediction on test data

In [13]:
nlp_model = load_model('nlp_model')

### Reading a pdf resume and converting into text for prediction

In [16]:

fname = r"C:\Users\Ravi kumar\Downloads\Resume-and-CV-Summarization-and-Parsing-with-Spacy-in-Python-master\Resume-and-CV-Summarization-and-Parsing-with-Spacy-in-Python-master\Alice Clark CV.pdf"
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text+str(page.get_text())

tx = " ".join(text.split("\n"))

## Model prediction & converying output in json to use for UI team

In [15]:
prediction(nlp_model,tx)

NAME                          -Alice Clark
LOCATION                      -Delhi
COMPANIES WORKED AT           -Microsoft
DESIGNATION                   -Software Engineer
COMPANIES WORKED AT           -Microsoft
COMPANIES WORKED AT           -Microsoft
COMPANIES WORKED AT           -Microsoft
COMPANIES WORKED AT           -Microsoft
COMPANIES WORKED AT           -Microsoft
COMPANIES WORKED AT           -Microsoft
COMPANIES WORKED AT           -Microsoft
COLLEGE NAME                  -Indian Institute of Technology – Mumbai


{0: {'NAME': 'Alice Clark',
  'LOCATION': 'Delhi',
  'COMPANIES WORKED AT': 'Microsoft',
  'DESIGNATION': 'Software Engineer',
  'COLLEGE NAME': 'Indian Institute of Technology – Mumbai'}}

### This is way to extract ,mail from text if model will not able to extract 

In [17]:
import re

regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')

def isValid(email):
    if re.fullmatch(regex, email):
        print("Valid email",re.fullmatch(regex, email).group())
    else:
        print("Invalid email")

In [25]:
isValid('ravi@scaledge.com')

Valid email ravi@scaledge.com


In [26]:
isValid('ravi@gmail.com')

Valid email ravi@gmail.com


In [27]:
isValid('ravi@gmail.co')

Valid email ravi@gmail.co


In [28]:
isValid('ravigmail.co')

Invalid email


In [29]:
isValid('ravi@gmail.co.in')

Valid email ravi@gmail.co.in
